## This notebook explores 1 record that was kicked back from Lex's R code after fixing. And it also explores 25 records kicked back from Lex. 
### The output of this notebook is:
#### 1. 1 + 24 Lat/Long calculated for FAA Airport Identifier
#### 2. 1  records back to Lex to fix IDENT

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import geopy
from geopy.distance import geodesic

In [3]:
# Read in csv file that contains one record
nonstd = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Olga_Non-Standard_1OUTOF412 -- RETURNED.csv')
nonstd

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,DATASET,UASLOCATION,RWYLOCATION
0,SEA ATCT advised UAS activity reported by King...,11/20/2018,3:01,2018-11-20T03:01Z,NaN,DEN,UAS,NaN,SEA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT,.25 NM SW of SEA,NaN


In [6]:
# Extract IDENT
uas_loc = nonstd['UASLOCATION'].tolist()
uas_airport = []

for i in range(len(uas_loc)):
    airport = uas_loc[i].split(' ')[-1]
    
    if len(airport) <= 4:
        uas_airport.append(airport)
    else:
        trim_air = airport[-3:]
        uas_airport.append(trim_air)
        
uas_airport_df = pd.DataFrame()
uas_airport_df['IDENT'] = uas_airport
uas_airport_df

,IDENT
0,SEA


In [4]:
# Read in airports dataset
airports = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/airports_cleaned.csv')

In [7]:
# Merge dataframe with airports dataset
uas_air_loc= pd.merge(uas_airport_df, airports, on='IDENT', how='left')
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,SEA,Seattle-Tacoma Intl,47.449889,-122.311778,KSEA


In [8]:
# Add needed columns
add_col = nonstd['REMARKS'].tolist()
uas_air_loc['REMARKS'] = add_col
add_col1 = nonstd['UASLOCATION'].tolist()
uas_air_loc['UASLOCATION'] = add_col1
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
0,SEA,Seattle-Tacoma Intl,47.449889,-122.311778,KSEA,SEA ATCT advised UAS activity reported by King...,.25 NM SW of SEA


In [9]:
# Create Dictionary of Bearings and Respective Degrees
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
}

# Separate distnace and bearing into two columns
uas_loc = uas_air_loc['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[0:])
    bearings.append(bearing_abrev)

# Create DataFrame to store UAS Location Information
new_df = pd.DataFrame(uas_air_loc['REMARKS'])
new_df['UAS_LOC'] = uas_air_loc['UASLOCATION']
new_df['IDENT'] = uas_air_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing
0,SEA ATCT advised UAS activity reported by King...,.25 NM SW of SEA,SEA,.25,SW


In [10]:
# Convert distance from NM to KM and bearing to degrees
new_df['Distance'] = pd.to_numeric(new_df['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['Airport_Latitude'] = uas_air_loc['LATITUDE']
new_df['Airport_Longitude'] = uas_air_loc['LONGITUDE']
new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude
0,SEA ATCT advised UAS activity reported by King...,.25 NM SW of SEA,SEA,0.25,SW,0.463,225,47.449889,-122.311778


In [11]:
# Merge, drop and rename columns
new_df2 = pd.merge(new_df, airports, on='IDENT', how='left')
new_df2 = new_df2.drop(['Airport_Latitude', 'Airport_Longitude'], 1)
new_df2 = new_df2.rename({'LONGITUDE': 'Airport_Longitude', 'LATITUDE': 'Airport_Latitude'}, axis=1)
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,SEA ATCT advised UAS activity reported by King...,.25 NM SW of SEA,SEA,0.25,SW,0.463,225,Seattle-Tacoma Intl,47.449889,-122.311778,KSEA


In [12]:
# Calculate UAS lat/long
uas_lat = []
uas_long = []

for i in range(len(new_df)):
    lat_airport = pd.to_numeric(new_df2['Airport_Latitude'][i])
    long_airport = pd.to_numeric(new_df2['Airport_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_airport, long_airport)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
new_df2['UAS_Latitude'] = uas_lat
new_df2['UAS_Longitude'] = uas_long
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID,UAS_Latitude,UAS_Longitude
0,SEA ATCT advised UAS activity reported by King...,.25 NM SW of SEA,SEA,0.25,SW,0.463,225,Seattle-Tacoma Intl,47.449889,-122.311778,KSEA,47.446944,-122.316119


In [13]:
# Export record to csv file
new_df2 = new_df2.drop(['Distance','Bearing','Distance_Kilometers','Bearing_Degrees','ICAO_ID'], 1)
new_df2 = new_df2.rename({'Airport_Longitude': 'Origin_Longitude', 'Airport_Latitude': 'Origin_Latitude'}, axis=1)
new_df2 = pd.merge(new_df2, nonstd, on='REMARKS', how='left')
new_df2.to_csv('COMPLETED-Non-std-1Airport.csv', index = False)

In [ ]:
###################################################################################################################

In [14]:
# Read in the csv file that contains 25 records
nonstd1 = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Olga_All_Points_Non-Standard_25OUTOF1280.csv')
nonstd1

,REMARKS,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,MORID.CEDAR,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION,DATASET
0,Reported a uas at the same altitude while ESE ...,3/28/2018,5:50,2018-03-28T05:50Z,MHO35,DEN,UAS,NaN,HNL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6 NM SE of HNL,AIRPORT
1,TSA NCRCC reports Walter Reed will be conducti...,9/4/2018,21:51,2018-09-04T21:51Z,NaN,NCRCC,NCR-COORDINATION,"OTHER, UAS",NCRCC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,012 NM 355 of DCA,FIXED RADIAL
2,Reported a uas at 700 feet .5 E of PMP. No eva...,9/22/2018,22:02,2018-09-22T22:02Z,N58LU,DEN,UAS,NaN,PMP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM E of PMP,AIRPORT
3,Reported a uas left side at same altitude whil...,12/4/2018,3:50,2018-12-04T03:50Z,DAL1287,CONR,UAS,NaN,LGA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 NM NE LGA,AIRPORT
4,"Reported # uas 1,000 feet below aircraft while...",1/1/2019,23:35,2019-01-01T23:35Z,JBU2953,DEN,UAS,NaN,PBI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 NM NW of PBI,AIRPORT
5,"Reported passing over a uas at 11,000 feet whi...",3/18/2019,4:18,2019-03-18T04:18Z,AAL2399,DEN,UAS,NaN,D10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50 NM SW of DFW,AIRPORT
6,"FBI uas operation at the ADW182003.5, AOB 200 ...",4/4/2019,19:40,2019-04-04T19:40Z,NaN,NCRCC,UAS,NCR-COORDINATION,NCRCC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,003 NM 182 of ADW,FIXED RADIAL
7,Reported a shiny uas 500 feet above from the 1...,6/12/2019,21:24,2019-06-12T21:24Z,UAL948,DEN,UAS,NaN,NCT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 NM SE OAK,AIRPORT
8,uas:Two aircraft reported a uas at approximate...,6/23/2019,22:13,2019-06-23T22:13Z,CMB303,NCRCC,UAS,NaN,PCT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12 NM E of BWI,AIRPORT
9,"Reported a black uas at 2,000 feet, no directi...",6/23/2019,23:39,2019-06-23T23:39Z,N434MC,NCRCC,UAS,NaN,PCT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,029 NM 264 of DCA,FIXED RADIAL


In [15]:
# Extract IDENTs
uas_loc = nonstd1['UASLOCATION'].tolist()
uas_airport = []

for i in range(len(uas_loc)):
    airport = uas_loc[i].split(' ')[-1]
    
    if len(airport) <= 4:
        uas_airport.append(airport)
    else:
        trim_air = airport[-3:]
        uas_airport.append(trim_air)
        
uas_airport_df = pd.DataFrame()
uas_airport_df['IDENT'] = uas_airport
uas_airport_df

,IDENT
0,HNL
1,DCA
2,PMP
3,LGA
4,PBI
5,DFW
6,ADW
7,OAK
8,BWI
9,DCA


In [16]:
# Merge Idents datarame with airports
uas_air_loc= pd.merge(uas_airport_df, airports, on='IDENT', how='left')
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,HNL,Daniel K Inouye Intl,21.317817,-157.920227,PHNL
1,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
2,PMP,Pompano Beach Airpark,26.247401,-80.111208,KPMP
3,LGA,Laguardia,40.777250,-73.872611,KLGA
4,PBI,Palm Beach Intl,26.683162,-80.095592,KPBI
5,DFW,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW
6,ADW,Joint Base Andrews,38.810826,-76.867046,KADW
7,OAK,Metro Oakland Intl,37.721250,-122.221139,KOAK
8,BWI,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI
9,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA


In [17]:
# Add needed columns
add_col = nonstd1['REMARKS'].tolist()
uas_air_loc['REMARKS'] = add_col
add_col1 = nonstd1['UASLOCATION'].tolist()
uas_air_loc['UASLOCATION'] = add_col1
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
0,HNL,Daniel K Inouye Intl,21.317817,-157.920227,PHNL,Reported a uas at the same altitude while ESE ...,6 NM SE of HNL
1,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,TSA NCRCC reports Walter Reed will be conducti...,012 NM 355 of DCA
2,PMP,Pompano Beach Airpark,26.247401,-80.111208,KPMP,Reported a uas at 700 feet .5 E of PMP. No eva...,5 NM E of PMP
3,LGA,Laguardia,40.777250,-73.872611,KLGA,Reported a uas left side at same altitude whil...,3 NM NE LGA
4,PBI,Palm Beach Intl,26.683162,-80.095592,KPBI,"Reported # uas 1,000 feet below aircraft while...",10 NM NW of PBI
5,DFW,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,"Reported passing over a uas at 11,000 feet whi...",50 NM SW of DFW
6,ADW,Joint Base Andrews,38.810826,-76.867046,KADW,"FBI uas operation at the ADW182003.5, AOB 200 ...",003 NM 182 of ADW
7,OAK,Metro Oakland Intl,37.721250,-122.221139,KOAK,Reported a shiny uas 500 feet above from the 1...,3 NM SE OAK
8,BWI,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI,uas:Two aircraft reported a uas at approximate...,12 NM E of BWI
9,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"Reported a black uas at 2,000 feet, no directi...",029 NM 264 of DCA


In [19]:
# Find recrod that did not link to airports
uas_air_loc1 = uas_air_loc[uas_air_loc['NAME'].isnull()]
uas_air_loc1

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
13,ALT,NaN,NaN,NaN,NaN,FBI detected a uas at the JFK 300002.5 ALT 11...,5 ALT


In [20]:
# Link it back to original dataset
uas_air_loc1 = uas_air_loc1.drop(['NAME', 'LATITUDE', 'LONGITUDE', 'ICAO_ID'], axis=1)
uas_air_loc1 = pd.merge(uas_air_loc1, nonstd1, on='REMARKS', how='left')
uas_air_loc1

,IDENT,REMARKS,UASLOCATION_x,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,SECONDARYCODES,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION_y,DATASET
0,ALT,FBI detected a uas at the JFK 300002.5 ALT 11...,5 ALT,9/23/2019,18:47,2019-09-23T18:47Z,NaN,JATOC,UAS,TOI-UAS-TFR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 ALT,AIRPORT


In [29]:
# Save the remaining recrod and send back to Lex
uas_air_loc1.to_csv('1Remaining-Olga_All_Points_Non-Standard_25OUTOF1280.csv', index = False)

In [21]:
# Save the matching records into new dataframe
uas_air_loc = uas_air_loc[uas_air_loc['NAME'].notnull()]
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
0,HNL,Daniel K Inouye Intl,21.317817,-157.920227,PHNL,Reported a uas at the same altitude while ESE ...,6 NM SE of HNL
1,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,TSA NCRCC reports Walter Reed will be conducti...,012 NM 355 of DCA
2,PMP,Pompano Beach Airpark,26.247401,-80.111208,KPMP,Reported a uas at 700 feet .5 E of PMP. No eva...,5 NM E of PMP
3,LGA,Laguardia,40.777250,-73.872611,KLGA,Reported a uas left side at same altitude whil...,3 NM NE LGA
4,PBI,Palm Beach Intl,26.683162,-80.095592,KPBI,"Reported # uas 1,000 feet below aircraft while...",10 NM NW of PBI
5,DFW,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,"Reported passing over a uas at 11,000 feet whi...",50 NM SW of DFW
6,ADW,Joint Base Andrews,38.810826,-76.867046,KADW,"FBI uas operation at the ADW182003.5, AOB 200 ...",003 NM 182 of ADW
7,OAK,Metro Oakland Intl,37.721250,-122.221139,KOAK,Reported a shiny uas 500 feet above from the 1...,3 NM SE OAK
8,BWI,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI,uas:Two aircraft reported a uas at approximate...,12 NM E of BWI
9,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"Reported a black uas at 2,000 feet, no directi...",029 NM 264 of DCA


In [22]:
uas_air_loc.to_csv('24Airports.csv', index = False)
uas_air_loc = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/24Airports.csv')
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
0,HNL,Daniel K Inouye Intl,21.317817,-157.920227,PHNL,Reported a uas at the same altitude while ESE ...,6 NM SE of HNL
1,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,TSA NCRCC reports Walter Reed will be conducti...,012 NM 355 of DCA
2,PMP,Pompano Beach Airpark,26.247401,-80.111208,KPMP,Reported a uas at 700 feet .5 E of PMP. No eva...,5 NM E of PMP
3,LGA,Laguardia,40.777250,-73.872611,KLGA,Reported a uas left side at same altitude whil...,3 NM NE LGA
4,PBI,Palm Beach Intl,26.683162,-80.095592,KPBI,"Reported # uas 1,000 feet below aircraft while...",10 NM NW of PBI
5,DFW,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,"Reported passing over a uas at 11,000 feet whi...",50 NM SW of DFW
6,ADW,Joint Base Andrews,38.810826,-76.867046,KADW,"FBI uas operation at the ADW182003.5, AOB 200 ...",003 NM 182 of ADW
7,OAK,Metro Oakland Intl,37.721250,-122.221139,KOAK,Reported a shiny uas 500 feet above from the 1...,3 NM SE OAK
8,BWI,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI,uas:Two aircraft reported a uas at approximate...,12 NM E of BWI
9,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,"Reported a black uas at 2,000 feet, no directi...",029 NM 264 of DCA


In [23]:
#Create Dictionary of Bearings and Respective Degrees
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
}

# Separate distances and bearings into two columns
uas_loc = uas_air_loc['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[0:])
    bearings.append(bearing_abrev)

# Create DataFrame to store UAS Location Information
new_df = pd.DataFrame(uas_air_loc['REMARKS'])
new_df['UAS_LOC'] = uas_air_loc['UASLOCATION']
new_df['IDENT'] = uas_air_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing
0,Reported a uas at the same altitude while ESE ...,6 NM SE of HNL,HNL,6,SE
1,TSA NCRCC reports Walter Reed will be conducti...,012 NM 355 of DCA,DCA,012,355
2,Reported a uas at 700 feet .5 E of PMP. No eva...,5 NM E of PMP,PMP,5,E
3,Reported a uas left side at same altitude whil...,3 NM NE LGA,LGA,3,NE
4,"Reported # uas 1,000 feet below aircraft while...",10 NM NW of PBI,PBI,10,NW
5,"Reported passing over a uas at 11,000 feet whi...",50 NM SW of DFW,DFW,50,SW
6,"FBI uas operation at the ADW182003.5, AOB 200 ...",003 NM 182 of ADW,ADW,003,182
7,Reported a shiny uas 500 feet above from the 1...,3 NM SE OAK,OAK,3,SE
8,uas:Two aircraft reported a uas at approximate...,12 NM E of BWI,BWI,12,E
9,"Reported a black uas at 2,000 feet, no directi...",029 NM 264 of DCA,DCA,029,264


In [28]:
# Convert distance from NM to KM
new_df['Distance'] = pd.to_numeric(new_df['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['Airport_Latitude'] = uas_air_loc['LATITUDE']
new_df['Airport_Longitude'] = uas_air_loc['LONGITUDE']
new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude
0,Reported a uas at the same altitude while ESE ...,6 NM SE of HNL,HNL,6,SE,11.112,135,21.317817,-157.920227
1,TSA NCRCC reports Walter Reed will be conducti...,012 NM 355 of DCA,DCA,12,355,22.224,355,38.851444,-77.037722
2,Reported a uas at 700 feet .5 E of PMP. No eva...,5 NM E of PMP,PMP,5,E,9.260,90,26.247401,-80.111208
3,Reported a uas left side at same altitude whil...,3 NM NE LGA,LGA,3,NE,5.556,45,40.777250,-73.872611
4,"Reported # uas 1,000 feet below aircraft while...",10 NM NW of PBI,PBI,10,NW,18.520,315,26.683162,-80.095592
5,"Reported passing over a uas at 11,000 feet whi...",50 NM SW of DFW,DFW,50,SW,92.600,225,32.897233,-97.037695
6,"FBI uas operation at the ADW182003.5, AOB 200 ...",003 NM 182 of ADW,ADW,3,182,5.556,182,38.810826,-76.867046
7,Reported a shiny uas 500 feet above from the 1...,3 NM SE OAK,OAK,3,SE,5.556,135,37.721250,-122.221139
8,uas:Two aircraft reported a uas at approximate...,12 NM E of BWI,BWI,12,E,22.224,90,39.175728,-76.668991
9,"Reported a black uas at 2,000 feet, no directi...",029 NM 264 of DCA,DCA,29,264,53.708,264,38.851444,-77.037722


In [29]:
# Merge, drop and rename columns
new_df2 = pd.merge(new_df, airports, on='IDENT', how='left')
new_df2 = new_df2.drop(['Airport_Latitude', 'Airport_Longitude'], 1)
new_df2 = new_df2.rename({'LONGITUDE': 'Airport_Longitude', 'LATITUDE': 'Airport_Latitude'}, axis=1)
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,Reported a uas at the same altitude while ESE ...,6 NM SE of HNL,HNL,6,SE,11.112,135,Daniel K Inouye Intl,21.317817,-157.920227,PHNL
1,TSA NCRCC reports Walter Reed will be conducti...,012 NM 355 of DCA,DCA,12,355,22.224,355,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA
2,Reported a uas at 700 feet .5 E of PMP. No eva...,5 NM E of PMP,PMP,5,E,9.260,90,Pompano Beach Airpark,26.247401,-80.111208,KPMP
3,Reported a uas left side at same altitude whil...,3 NM NE LGA,LGA,3,NE,5.556,45,Laguardia,40.777250,-73.872611,KLGA
4,"Reported # uas 1,000 feet below aircraft while...",10 NM NW of PBI,PBI,10,NW,18.520,315,Palm Beach Intl,26.683162,-80.095592,KPBI
5,"Reported passing over a uas at 11,000 feet whi...",50 NM SW of DFW,DFW,50,SW,92.600,225,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW
6,"FBI uas operation at the ADW182003.5, AOB 200 ...",003 NM 182 of ADW,ADW,3,182,5.556,182,Joint Base Andrews,38.810826,-76.867046,KADW
7,Reported a shiny uas 500 feet above from the 1...,3 NM SE OAK,OAK,3,SE,5.556,135,Metro Oakland Intl,37.721250,-122.221139,KOAK
8,uas:Two aircraft reported a uas at approximate...,12 NM E of BWI,BWI,12,E,22.224,90,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI
9,"Reported a black uas at 2,000 feet, no directi...",029 NM 264 of DCA,DCA,29,264,53.708,264,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA


In [30]:
# Calculate UAS lat/long
uas_lat = []
uas_long = []

for i in range(len(new_df)):
    lat_airport = pd.to_numeric(new_df2['Airport_Latitude'][i])
    long_airport = pd.to_numeric(new_df2['Airport_Longitude'][i])
    b = pd.to_numeric(new_df['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_airport, long_airport)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
new_df2['UAS_Latitude'] = uas_lat
new_df2['UAS_Longitude'] = uas_long
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID,UAS_Latitude,UAS_Longitude
0,Reported a uas at the same altitude while ESE ...,6 NM SE of HNL,HNL,6,SE,11.112,135,Daniel K Inouye Intl,21.317817,-157.920227,PHNL,21.246835,-157.844529
1,TSA NCRCC reports Walter Reed will be conducti...,012 NM 355 of DCA,DCA,12,355,22.224,355,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,39.050870,-77.060098
2,Reported a uas at 700 feet .5 E of PMP. No eva...,5 NM E of PMP,PMP,5,E,9.260,90,Pompano Beach Airpark,26.247401,-80.111208,KPMP,26.247371,-80.018522
3,Reported a uas left side at same altitude whil...,3 NM NE LGA,LGA,3,NE,5.556,45,Laguardia,40.777250,-73.872611,KLGA,40.812618,-73.826048
4,"Reported # uas 1,000 feet below aircraft while...",10 NM NW of PBI,PBI,10,NW,18.520,315,Palm Beach Intl,26.683162,-80.095592,KPBI,26.801293,-80.227300
5,"Reported passing over a uas at 11,000 feet whi...",50 NM SW of DFW,DFW,50,SW,92.600,225,Dallas-Fort Worth Intl,32.897233,-97.037695,KDFW,32.304859,-97.732938
6,"FBI uas operation at the ADW182003.5, AOB 200 ...",003 NM 182 of ADW,ADW,3,182,5.556,182,Joint Base Andrews,38.810826,-76.867046,KADW,38.760807,-76.869277
7,Reported a shiny uas 500 feet above from the 1...,3 NM SE OAK,OAK,3,SE,5.556,135,Metro Oakland Intl,37.721250,-122.221139,KOAK,37.685845,-122.176599
8,uas:Two aircraft reported a uas at approximate...,12 NM E of BWI,BWI,12,E,22.224,90,Baltimore/Washington Intl Thurgood Marshall,39.175728,-76.668991,KBWI,39.175445,-76.411804
9,"Reported a black uas at 2,000 feet, no directi...",029 NM 264 of DCA,DCA,29,264,53.708,264,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.799254,-77.652592


In [31]:
# Export records to csv file
new_df2 = new_df2.drop(['Distance','Bearing','Distance_Kilometers','Bearing_Degrees','ICAO_ID'], 1)
new_df2 = new_df2.rename({'Airport_Longitude': 'Origin_Longitude', 'Airport_Latitude': 'Origin_Latitude'}, axis=1)
new_df2 = pd.merge(new_df2, nonstd1, on='REMARKS', how='left')
new_df2.to_csv('COMPLETED-Non-std-24Airports.csv', index = False)

In [ ]:
###################################################################################################################